### PyTorch

Note that this is actually a numpy-based implementation that matches exactly with the computation in the original PyTorch implementation. To verify, compare with [Pixel Shuffle Demo 1 Google Colab notebook](https://colab.research.google.com/drive/1C7W51LvcTccRxKJGZ5m61DDzR4P6fW1T?usp=sharing) or check the notebook of the same name in my [github repository](https://github.com/iamshnoo/mlpack-testing).

In [1]:
## Imports
import torch
import numpy as np

In [2]:
## User Input
batch_size = 1     
channels = 4   
upscale_factor = 2     
height = 2         
width = 2         

input = torch.zeros([batch_size, channels, height, width])
v = np.arange(1,height*width + 1)
v = v.reshape(height,width)
input[0][0] = torch.from_numpy(v) 

In [3]:
print('INPUT for Pixel Shuffle: \n')
print(input)

INPUT for Pixel Shuffle: 

tensor([[[[1., 2.],
          [3., 4.]],

         [[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]]])


In [4]:
## Forward()
output = torch.zeros([batch_size, channels // (upscale_factor **2), height * upscale_factor, width * upscale_factor])
for n in range(output.size(0)):
  for c in range(output.size(1)):
      for h in range(output.size(2)):
          for w in range(output.size(3)):
              height_idx = h // upscale_factor
              weight_idx = w // upscale_factor
              channel_idx = (upscale_factor * (h % upscale_factor)) + (w % upscale_factor) + (c * upscale_factor ** 2)
              output[n, c, h, w] = input[n, channel_idx, height_idx, weight_idx]
            
## this is the output from Forward() of Pixel Shuffle
print('OUTPUT from Pixel Shuffle: \n')
print(output)

OUTPUT from Pixel Shuffle: 

tensor([[[[1., 0., 2., 0.],
          [0., 0., 0., 0.],
          [3., 0., 4., 0.],
          [0., 0., 0., 0.]]]])


In [5]:
## this is gy for Pixel Shuffle layer simulated as a tensor filled with an arbitrary sequence of values.
a = height * upscale_factor * width * upscale_factor + 1
a = float(a)
gy = torch.arange(1.,a).reshape(output.shape)
print('Hypothetical gy for Pixel Shuffle: \n')
print(gy)

Hypothetical gy for Pixel Shuffle: 

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])


In [6]:
## Backward()
g = torch.zeros(input.shape)
for n in range(gy.size(0)):
  for c in range(gy.size(1)):
      for h in range(gy.size(2)):
          for w in range(gy.size(3)):
              height_idx = h // upscale_factor
              weight_idx = w // upscale_factor
              channel_idx = (upscale_factor * (h % upscale_factor)) + (w % upscale_factor) + (c * upscale_factor ** 2)
              g[n, channel_idx, height_idx, weight_idx] = gy[n, c, h, w] 

## this is g for Pixel Shuffle layer
print('g for Pixel Shuffle: \n')
print(g)

g for Pixel Shuffle: 

tensor([[[[ 1.,  3.],
          [ 9., 11.]],

         [[ 2.,  4.],
          [10., 12.]],

         [[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])


### mlpack

Here I give 2 images in the batch, one of which is the same as above, to help verify that the calculaions are correct <br> and also simultaneously to check the support for batching.

In [7]:
%%capture
!sudo apt-get install libmlpack-dev 

In [8]:
%%capture
%%writefile test.cpp
#include <iostream>
#include <armadillo>
#define PRINT 1

using namespace std;
using namespace arma;

int main()
{
  // User input
  arma::mat input;
  input << 1 << 3 << 2 << 4 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << endr
        << 5 << 7 << 6 << 8 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << 0 << endr;
  input = input.t();

  size_t size = 4; // input channels
  size_t upscale_factor = 2;
  size_t height = 2;
  size_t width = 2;

  // Forward()
  size_t batchSize = input.n_cols;
  size_t size_out = size / std::pow(upscale_factor, 2); // output channels
  size_t output_height = height * upscale_factor;
  size_t output_width = width * upscale_factor;

  arma::mat output;
  output.zeros(output_height * output_width * size_out, batchSize );

  for(size_t n = 0; n < batchSize; n++)
  {
    arma::mat inputImage = input.col(n);
    arma::mat outputImage = output.col(n);
    arma::cube inputTemp(const_cast<arma::mat&>(inputImage).memptr(), height, width, size, false, false);
    arma::cube outputTemp(const_cast<arma::mat&>(outputImage).memptr(), output_height, output_width, size_out, false, false);

    for (size_t c = 0; c < size_out ; c++)
    {
      for (size_t h = 0; h < output_height; h++)
      {
        for (size_t w = 0; w < output_width; w++)
        {
          size_t height_index = h / upscale_factor;
          size_t width_index = w / upscale_factor;
          size_t channel_index = (upscale_factor * (h % upscale_factor)) + (w % upscale_factor) + (c * std::pow(upscale_factor, 2));
          outputTemp(w, h, c) = inputTemp(width_index, height_index, channel_index);
        }
      }
    }

    output.col(n) = outputImage;

    if(PRINT)
    {
      cout << "Image " << n << " calculations: " << endl;
      cout << "-----------------------------------" << endl;
      cout << "-----------------------------------" << endl;
      cout << "INPUT for Pixel Shuffle: " << endl;
      cout << inputTemp << endl;
      cout << "-----------------------------------" << endl;
      cout << "OUTPUT from Pixel Shuffle: " << endl;
      cout << outputTemp << endl;
      cout << "-----------------------------------" << endl;
    }

  }

  // this is gy for Pixel Shuffle layer simulated as a tensor filled with an arbitrary sequence of values.
  arma::mat gy;
  gy << 1 << 5 << 9 << 13 << 2 << 6 << 10 << 14 << 3 << 7 << 11 << 15 << 4 << 8 << 12 << 16 << endr
    << 17 << 21 << 25 << 29 << 18 << 22 << 26 << 30 << 19 << 23 << 27 << 31 << 20 << 24 << 28 << 32 << endr;
  gy = gy.t();

  // Backward()
  arma::mat g;
  g.zeros(arma::size(input));

  for(size_t n = 0; n < batchSize; n++)
  {
    arma::mat gyImage = gy.col(n);
    arma::mat gImage = g.col(n);
    arma::cube gyTemp(const_cast<arma::mat&>(gyImage).memptr(), output_height, output_width, size_out, false, false);
    arma::cube gTemp(const_cast<arma::mat&>(gImage).memptr(), height, width, size, false, false);

    for (size_t c = 0; c < size_out ; c++)
    {
      for (size_t h = 0; h < output_height; h++)
      {
        for (size_t w = 0; w < output_width; w++)
        {
          size_t height_index = h / upscale_factor;
          size_t width_index = w / upscale_factor;
          size_t channel_index = (upscale_factor * (h % upscale_factor)) + (w % upscale_factor) + (c * std::pow(upscale_factor, 2));
          gTemp(width_index, height_index, channel_index) = gyTemp(w, h, c);
        }
      }
    }

    g.col(n) = gImage;

    if (PRINT)
    {
      cout << "Image " << n << " calculations: " << endl;
      cout << "-----------------------------------" << endl;
      cout << "Hypothetical gy for Pixel Shuffle: " << endl;
      cout << gyTemp << endl;
      cout << "-----------------------------------" << endl;
      cout << "g for Pixel Shuffle: " << endl;
      cout << gTemp << endl;
      cout << "-----------------------------------" << endl;
    }
  }

  return 0;
}

In [9]:
%%script bash
g++ test.cpp -o test -larmadillo && ./test

Image 0 calculations: 
-----------------------------------
-----------------------------------
INPUT for Pixel Shuffle: 
[cube slice 0]
   1.0000   2.0000
   3.0000   4.0000

[cube slice 1]
        0        0
        0        0

[cube slice 2]
        0        0
        0        0

[cube slice 3]
        0        0
        0        0


-----------------------------------
OUTPUT from Pixel Shuffle: 
[cube slice 0]
   1.0000        0   2.0000        0
        0        0        0        0
   3.0000        0   4.0000        0
        0        0        0        0


-----------------------------------
Image 1 calculations: 
-----------------------------------
-----------------------------------
INPUT for Pixel Shuffle: 
[cube slice 0]
   5.0000   6.0000
   7.0000   8.0000

[cube slice 1]
        0        0
        0        0

[cube slice 2]
        0        0
        0        0

[cube slice 3]
        0        0
        0        0


-----------------------------------
OUTPUT from Pixel Shuff